In [65]:

from dolfinx import mesh as msh, fem, default_scalar_type, la
from mpi4py import MPI
import ufl
import numpy as np

import scipy

nx=128
ny=128
comm = MPI.COMM_WORLD
cell_type=msh.CellType.quadrilateral
domain = msh.create_unit_square(comm=comm, nx=nx, ny=ny, cell_type=cell_type)

Wh = fem.functionspace(domain, ('P', 1))
uh = fem.Function(Wh)
domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
boundary_facets = msh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(Wh, domain.topology.dim-1, boundary_facets)
eps = fem.Constant(domain, default_scalar_type(1e-8))
b = ufl.as_vector((fem.Constant(domain, default_scalar_type(1.0)),fem.Constant(domain, default_scalar_type(0.0))))
c = fem.Constant(domain, default_scalar_type(0.0))
f = fem.Constant(domain, default_scalar_type(1.0))
bcs = [fem.dirichletbc(fem.Constant(domain, default_scalar_type(0.0)), boundary_dofs, Wh)]
g = fem.Constant(domain, default_scalar_type(0.0))


_u= ufl.TrialFunction(Wh)
_v = ufl.TestFunction(Wh)

#FEM space for the SUPG-parameters/weights
Yh = fem.functionspace(domain, ("DG", 0))
yh = fem.Function(Yh)

#number of locally owned cells
_owned_cells = yh.x.index_map.size_local

#SUPG-forms
_a = eps * ufl.dot(ufl.grad(_u), ufl.grad(_v)) * ufl.dx + ufl.dot(b, ufl.grad(_u)) * _v * ufl.dx
sh_test = (yh * ufl.dot(b, ufl.grad(_v))) 
_sh = -eps * ufl.div(ufl.grad(_u)) * sh_test * ufl.dx + ufl.dot(b, ufl.grad(_u)) * sh_test * ufl.dx
if c != None:
    _sh += c * _u* sh_test * ufl.dx
    _a +=  c * _u* _v * ufl.dx

_L = f * _v * ufl.dx
if g is not None:
    _L += g * _v * ufl.ds
_rh = f * yh * ufl.dot(b, ufl.grad(_v)) * ufl.dx


A = fem.petsc.assemble_matrix(fem.form(_a))
Sh = fem.petsc.assemble_matrix(fem.form(_sh))
L = fem.petsc.assemble_vector(fem.form(_L))
Rh = fem.petsc.assemble_vector(fem.form(_rh))

A.assemble()
Sh.assemble()
L.assemble()
Rh.assemble()
for bc in bcs:
    dofs, _ = bc._cpp_object.dof_indices()
    A.zeroRowsLocal(dofs, diag=1)
    Sh.zeroRowsLocal(dofs, diag=0)

In [66]:
Rh_mat=[]
Sh_cube=[]
num_cells = len(yh.x.array)
for c in range(num_cells):
    yh.x.array[:] = 0
    yh.x.array[c] = 1
    Rh = fem.petsc.assemble_vector(fem.form(_rh))
    Rh.assemble()
    Sh = fem.petsc.assemble_matrix(fem.form(_sh))
    Sh.assemble()
    for bc in bcs:
        dofs, _ = bc._cpp_object.dof_indices()
        Sh.zeroRowsLocal(dofs, diag=0)
    Rh_mat.append(Rh.array)
    Sh_cube.append(Sh)

Rh_mat = np.array(Rh_mat)

In [69]:
Sh_cube[0].getValuesCSR()[2]

array([0., 0., 0., ..., 0., 0., 0.], shape=(148225,))